# Automatic classification of 200+ bird species

## Prepare environment

First, download last version of fastai.

In [ ]:
!pip3 install -q fastai==2.0.16

Then import it.

In [ ]:
from fastai.vision.all import *

Specify our working directory, i.e. the one containing the dataset.

In [ ]:
working_directory = "/kaggle/input/100-bird-species"

We are now ready to load the images and apply transformations to them.

## Load and transform images

Fastai allows us to load dataset with a lot of different methods (using pandas dataframes, regex, folder, ...). 
Here we'll load our images from folder since the dataset structure is already well formated.
We'll first enlarge all our images to $340*340$ and then randomly crop them to size $224*224$. This way, we'll generate more diverse training images, all having at least parts of the bird shown.

In [ ]:
dls = ImageDataLoaders.from_folder(working_directory, 
                 item_tfms=Resize(340),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75), bs=64)
dls.show_batch(nrows=3, ncols=3)

We observe that the images seems to be correctly loaded and transformed.

## Training

We load pretrained resnet18.

In [ ]:
learn = cnn_learner(dls, resnet18, pretrained=True, metrics=accuracy, model_dir="/kaggle/working")

As first try, we decide to take advantage of `fine_tune()` method, using the default parameters. From experience, it appears to be a good place to start for many different problems.

In [ ]:
callbacks_to_use = [SaveModelCallback(comp=np.greater, monitor='accuracy', fname="best_valid_accuracy")]

In [ ]:
learn.fine_tune(15, cbs=callbacks_to_use)

Accuracy is already very high. We decide to stop experimenting since the possible improvements relative to the time spent are not worth it.
We however will have a quick visual check about the results to be sure there are no obvious errors:

In [ ]:
learn.show_results()

Alright, we'll now export our model.

In [ ]:
learn.path = Path("/kaggle/working/")
learn.export('2020-10-27_1.6.0-birds-classifier-resnet18-224.pkl')

## Test evaluation
Let's now evaluate the test set:

In [ ]:
import torch
import os

test_dir = working_directory + "/test/"
list_dir = os.listdir(test_dir)
classes = learn.dls.categorize.vocab.items

test_predictions = []
target_predictions= []


for folder in list_dir:
  list_images = os.listdir(test_dir + folder + "/")
  for image_name in list_images:
    target_predictions.append(folder)
    prediction, _, _ = learn.predict(test_dir + folder + "/" + image_name)
    test_predictions.append(prediction)

In [ ]:
correct = 0
total = 0

for element_index in range(len(test_predictions)):
  total += 1
  if target_predictions[element_index] == test_predictions[element_index]:
    correct += 1

print(f"Test set accuracy: {100*correct/total}")